In [7]:
from monet_utils import *
import torch
from token_generator import token_gen

In [8]:
#Parameter Space 
#----------------------------------------------------------------------------------------------------------------------------------
#Backbone 
patch_size = 16
arch = 'dino_small'
scale = [patch_size, patch_size]
device = 'cuda:2' if torch.cuda.is_available() else 'cpu'
#----------------------------------------------------------------------------------------------------------------------------------
#Dataset param
data = 'COCO20k'
data_set = 'train'
remove_hard = True
masking_ratio = None
masking_size = None
resize = None
crop = None
if data == 'COCO20k':
    root_path = './datasets/COCO/images/train2014'
elif data == 'VOC07':
    root_path = './datasets/VOC2007/VOCdevkit/VOC2007/JPEGImages'
elif data == 'VOC12':
    root_path = './datasets/VOC2012/VOCdevkit/VOC2012/JPEGImages'
#----------------------------------------------------------------------------------------------------------------------------------
#Kmeans param
k_cluster = 2
which_feature = 'output'
depth = -1 
random_seed = 100
n_iter = 200
cosine = True
if cosine:
    spherical = 'cosine'
else:
    spherical = 'l2'
#----------------------------------------------------------------------------------------------------------------------------------
#Batch Token Sep
#COCO20k는 2개의 파트로 나눔 
part = 1

In [9]:
model = get_model(arch=arch, patch_size=patch_size, device=device)

Downloading: "https://dl.fbaipublicfiles.com/dino/dino_deitsmall16_pretrain/dino_deitsmall16_pretrain.pth" to /root/.cache/torch/hub/checkpoints/dino_deitsmall16_pretrain.pth


  0%|          | 0.00/82.7M [00:00<?, ?B/s]

Pre-trained weights founde at dino_deitsmall16_pretrain/dino_deitsmall16_pretrain.pth and loaded with msg : <All keys matched successfully>


In [10]:
dataset, dataloader = dataset_load(dataset_name=data, 
                                   dataset_set=data_set,
                                   remove_hards=False, 
                                   resize = (640,920),
                                   patch_size=patch_size)

loading annotations into memory...
Done (t=1.99s)
creating index...
index created!


In [11]:
#Mini Batch Token Gen
data_len = len(dataloader)
part_num = 2
batch_size = data_len//part_num
for part in range(1,part_num + 1):
    print(f'part number : {part}')
    if part == 1:
        start_num = 0
        end_num = start_num + batch_size
    elif part != 1 and part != part_num:
        start_num = end_num
        end_num = start_num + batch_size
    elif part == part_num:
        start_num = end_num
        end_num = data_len - 1
    print(f"{start_num} | {end_num}")
    token = token_gen(model=model,
                      dataset = dataset,
                      dataloader=dataloader,
                      root_path=root_path,
                      patch_size=patch_size,
                      start_num=start_num, end_num=end_num,
                      depth=-1, device=device)
    
    np.save(f'/data_hdd1/batch_token_{data}_{patch_size}_{arch}_{which_feature}_{part}_resize640',token)
    token = None
    if part == 1:
        break 

part number : 1
0 | 9908


100%|██████████| 9909/9909 [11:22<00:00, 14.52it/s]


In [5]:
#Part 1만 성공하고 2는 실패했을때
#Mini Batch Token Gen
data_len = len(dataloader)
part_num = 2
batch_size = data_len//part_num
for part in range(2,part_num + 1):
    start_num = 9908
    end_num = 19816
    print(f"{start_num} | {end_num}")
    token = token_gen(model=model,
                      dataset = dataset,
                      dataloader=dataloader,
                      root_path=root_path,
                      patch_size=patch_size,
                      start_num=start_num, end_num=end_num,
                      depth=-1, device=device)
    
    np.save(f'/data_hdd1/batch_token_{data}_{patch_size}_{arch}_{which_feature}_{part}',token)

9908 | 19816


  0%|          | 0/9909 [00:00<?, ?it/s]/root/anaconda3/envs/monet_faiss/lib/python3.8/site-packages/torch/nn/functional.py:3631: UserWarning: Default upsampling behavior when mode=bicubic is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  warnings.warn(
/root/anaconda3/envs/monet_faiss/lib/python3.8/site-packages/torch/nn/functional.py:3679: UserWarning: The default behavior for interpolate/upsample with float scale_factor changed in 1.6.0 to align with other frameworks/libraries, and now uses scale_factor directly, instead of relying on the computed output size. If you wish to restore the old behavior, please set recompute_scale_factor=True. See the documentation of nn.Upsample for details. 
  warnings.warn(
100%|██████████| 9909/9909 [29:39<00:00,  5.57it/s]  
